In [1]:
import os
import csv
import logging
import argparse
from tqdm import tqdm
from pathlib import Path
from datetime import datetime

import torch
from torch import optim
from torch.utils.data import random_split, DataLoader
from torch.optim.lr_scheduler import CyclicLR

from models.seg.UNet.unet_model import UNet, R2UNet
from models.seg.MaskRCNN.model import MaskRCNNResNet50

from utils.indicators import segmentation_indicators, detection_indicators
from utils.utils import EarlyStopping
from utils.data_loading import SegmentDatasetJson

In [2]:
image_dir = 'E:/Datas/work/HairEffect/RawData/HAM10000/HAM10000_images'
json_dir = '../datas/json_data/maskrcnn_annotation_HAM10000.json'

epochs = 200
batch_size = 4
num_classes = 2
gc = 5.0
lr = 1e-4
size = (256, 256)
val = 0.2
amp = True

device = torch.device('cuda' if torch.cuda.is_available()  else 'cpu')

In [3]:
now_time, now_h = datetime.now().strftime("%Y-%m-%d"), datetime.now().strftime("%H")
checkpoint_save_dir = Path(f'./checkpoints/MaskRCNN/{now_time}/{now_h}')
indicators_save_dir = Path(f'./indicators/MaskRCNN/{now_time}/{now_h}')
Path(checkpoint_save_dir).mkdir(parents=True, exist_ok=True)
Path(indicators_save_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
model = MaskRCNNResNet50(n_classes=num_classes)
model = model.to(device=device, memory_format=torch.channels_last)